In [1]:
import pandas as pd
from Bio import SeqIO


In [2]:
df = pd.read_csv('k.csv')

In [3]:
df.tail()

,Unnamed: 0,target_gene,lister
7477,7477,ENSG00000272573,[SeqRecord(seq=Seq('gtgtgaacacaactcactgtagcctc...
7478,7478,ENSG00000272617,[SeqRecord(seq=Seq('gtattttttgtagagagggggtcttg...
7479,7479,ENSG00000272772,[SeqRecord(seq=Seq('GCCAAACACCCTTTAAAATGAAACGT...
7480,7480,ENSG00000272822,[SeqRecord(seq=Seq('gccagtgagaaaagacaaagaaaagc...
7481,7481,ENSG00000273079,[SeqRecord(seq=Seq('ttgaactgaagtctgtcAGTTTAAAC...


In [4]:
b

In [5]:
aa = pd.read_csv('kmer4.csv')

In [6]:
df_merge = df.merge(aa,left_on = 'target_gene',right_on = 'target_gene')

In [7]:
k = df_merge[['target_gene','lister','size']]

In [8]:
k.head()

,target_gene,lister,size
0,ENSG00000000460,[SeqRecord(seq=Seq('AGCCAGAGAGGCCTCGGAAAGAGAAG...,68
1,ENSG00000000938,[SeqRecord(seq=Seq('ACTGTCAGTGTTCCTTTGTGTAAGAC...,39
2,ENSG00000000971,[SeqRecord(seq=Seq('GATTGTACATGAAAGTAGCTTAAATT...,45
3,ENSG00000001084,[SeqRecord(seq=Seq('CAGAGCATAAGGATGCCCATTAAGAT...,60
4,ENSG00000001167,[SeqRecord(seq=Seq('atgctttatacatgtgtgcactgagt...,13


In [9]:
k.iloc[0].lister

"[SeqRecord(seq=Seq('AGCCAGAGAGGCCTCGGAAAGAGAAGCCTACTTATTAAAAATAGCTTTCTggct...ggg'), id='chr1:169456879-169457109', name='chr1:169456879-169457109', description='chr1:169456879-169457109', dbxrefs=[]), SeqRecord(seq=Seq('AAACCTGGTTTTATTTGCAAAGCTTTAATCTTGATAAACGGGACCTTCTTAAAG...AAA'), id='chr1:169476826-169477196', name='chr1:169476826-169477196', description='chr1:169476826-169477196', dbxrefs=[]), SeqRecord(seq=Seq('GAATCTTGTGCAAGAACATAACAATTTTGTAAAAATGCAGTTGTTACACcttgc...ata'), id='chr1:169477483-169478174', name='chr1:169477483-169478174', description='chr1:169477483-169478174', dbxrefs=[]), SeqRecord(seq=Seq('ATTCTAGCTGCTAGGAGTGCAGAGAAGATTATTCATAATGCTGGCAAATATGAA...ATA'), id='chr1:169478295-169479018', name='chr1:169478295-169479018', description='chr1:169478295-169479018', dbxrefs=[]), SeqRecord(seq=Seq('TGCTGGAAGAAGAGAGAGGAGGAAGTTGAGGCCATAGAGAGGAAGGCCCTGAAA...TGG'), id='chr1:169482660-169483000', name='chr1:169482660-169483000', description='chr1:169482660-169483000', dbxrefs=[])

In [12]:
# sequences = k.iloc[0].lister
# SeqIO.write(sequences[0], "example.fasta", "fasta")

In [13]:
df_liver = pd.read_csv('liver_enh_to_gene_peakachuloop-linked.tsv', sep='\t')
s = df_liver.groupby('target_gene').size().reset_index()
s =s.sort_values(by=[0],ascending=False)


In [14]:
record = SeqIO.parse("test.fa.out", "fasta")
record_dict = SeqIO.to_dict(SeqIO.parse('test.fa.out', 'fasta'))
records = list(record)
pd_bed = pd.DataFrame.from_records([record_dict])
pd_bed  = pd_bed.transpose().reset_index()

In [15]:
p = s.merge(df_liver,left_on=['target_gene'],right_on=['target_gene'])
p['startEnd'] = p.apply(lambda row: row.enh_chrom +":" +str(row.enh_start)+"-"+str(row.enh_end), axis=1)
p = p.merge(pd_bed,left_on=['startEnd'],right_on=['index'])
p= p.rename(columns={'0_y': "lister"})
p = p[['startEnd', 'lister','target_gene']]
k= p.groupby('target_gene')['lister'].agg(list)
k = k.reset_index()

In [16]:
k.head()

,target_gene,lister
0,ENSG00000000460,"[(A, G, C, C, A, G, A, G, A, G, G, C, C, T, C,..."
1,ENSG00000000938,"[(A, C, T, G, T, C, A, G, T, G, T, T, C, C, T,..."
2,ENSG00000000971,"[(G, A, T, T, G, T, A, C, A, T, G, A, A, A, G,..."
3,ENSG00000001084,"[(C, A, G, A, G, C, A, T, A, A, G, G, A, T, G,..."
4,ENSG00000001167,"[(a, t, g, c, t, t, t, a, t, a, c, a, t, g, t,..."


In [17]:
sequences = k.iloc[1].lister
SeqIO.write(sequences, "example.fasta", "fasta")

39

In [18]:
sequences = k.iloc[1].lister
length = len(sequences)
print(sequences[0])

ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')


In [19]:
import numpy as np

In [20]:
a = np.zeros(shape=(length,length))

In [21]:
from Bio import pairwise2
from Bio.pairwise2 import format_alignment

In [22]:
def globalAlign(a,b):
    a =  a.seq
    b =  b.seq
    for k in pairwise2.align.globalms(a, b,2, -1, -.5, -.1):
        al1,al2, score, begin, end = k
    return score
   

In [23]:
for i in range(0,length):
    for j in range(0,length):
        print(sequences[i])
        print(sequences[j])
        a[i][j] = globalAlign(sequences[i],sequences[j])

ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27757569-27758308
Name: chr1:27757569-27758308
Description: chr1:27757569-2775830

ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27895128-27895824
Name: chr1:27895128-27895824
Description: chr1:27895128-27895824
Number of features: 0
Seq('AGATTGGTAGCACCTGGAGTGATGTCGCTAGGGGCGGGGTCGGGGCTTCGGACA...cct')
ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27895885-27896315
Name: chr1:27895885-27896315
Description: chr1:27895885-27896315
Number of features: 0
Seq('tgggtaaaatagcctgcctaagACAGGCAAGGATTTGAACCCCATGTTCTTCCA...TCC')
ID: chr1:27754030-27754337
Name: chr1:27754030-27754337
Description: chr1:27754030-27754337
Number of features: 0
Seq('ACTGTCAGTGTTCCTTTGTGTAAGACAGGCACTGGCAGGGTTTCACTTCTTCAT...TTG')
ID: chr1:27902457-27903165
Name: chr1:27902457-27903165
Description: chr1:27902457-2790316

ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27795251-27795506
Name: chr1:27795251-27795506
Description: chr1:27795251-27795506
Number of features: 0
Seq('aggccaaggagggtagatcacctgaggtcaggagttcaagaccagcctggccaa...ctg')
ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27796110-27796365
Name: chr1:27796110-27796365
Description: chr1:27796110-27796365
Number of features: 0
Seq('tcatataggagaaatcagaaaagtacctaaacataaaacaaggtaaaaagtgat...gaa')
ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27830263-27831052
Name: chr1:27830263-27831052
Description: chr1:27830263-2783105

ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27924338-27924752
Name: chr1:27924338-27924752
Description: chr1:27924338-27924752
Number of features: 0
Seq('tcagcctccccatctctctccttctccatctcagtcactctgtctcctttcttt...AGG')
ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27927285-27927635
Name: chr1:27927285-27927635
Description: chr1:27927285-27927635
Number of features: 0
Seq('TCCAACTTCCCTACTGTACACCTCCACTTAGCTCAGCAGGGGGTTCCAGAAGCC...AGC')
ID: chr1:27754772-27755454
Name: chr1:27754772-27755454
Description: chr1:27754772-27755454
Number of features: 0
Seq('TCTCTCCGAAGGCATGTTTGCTTTACTCCTCTTTGGGAAGAAAAGGGGTGGGGG...GAA')
ID: chr1:27900987-27902277
Name: chr1:27900987-27902277
Description: chr1:27900987-2790227

KeyboardInterrupt: 

In [ ]:
a

In [27]:
read = {}

In [28]:
def globalValue(df):
    length = len(df.lister)
    sequences = df['lister']
    b = np.zeros(shape=(length,length))
    name  = str(df.target_gene)
    for i in range(0,length-1):
        for j in range(0,length-1):
            if(b[j][i] != 0):
                b[i][j] = b[j][i]
            else:
                b[i][j] = globalAlign(sequences[i],sequences[j])
    read[name] = b
    return b

In [ ]:
np.array_equal(a, b)

In [25]:
from dask.dataframe import from_pandas

In [ ]:
ddf = from_pandas(k, npartitions=10)

In [29]:
for i in range(0,3):
    globalValue(k.iloc[i])

In [35]:
np.save('my_dict.npy',  read, allow_pickle=True)  

In [37]:
my_dict_back = np.load('my_dict.npy',allow_pickle=True)


In [38]:
my_dict_back

array({'ENSG00000000460': array([[ 460. ,   42.4,  322.3, ...,   54.6,   54.1,    0. ],
       [  42.4,  740. ,   -5.5, ...,  310.8,  319.9,    0. ],
       [ 322.3,   -5.5, 1382. , ...,    6.7,    5.4,    0. ],
       ...,
       [  54.6,  310.8,    6.7, ...,  464. ,  253.3,    0. ],
       [  54.1,  319.9,    5.4, ...,  253.3,  490. ,    0. ],
       [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ]]), 'ENSG00000000938': array([[ 614. ,  429.5,  453.2, ...,  353. ,  323.1,    0. ],
       [ 429.5, 1364. ,  735.7, ...,  494.5,  426.6,    0. ],
       [ 453.2,  735.7, 1478. , ...,  526.6,  446.5,    0. ],
       ...,
       [ 353. ,  494.5,  526.6, ...,  866. ,  347.5,    0. ],
       [ 323.1,  426.6,  446.5, ...,  347.5,  616. ,    0. ],
       [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ]]), 'ENSG00000000971': array([[1136. ,  476.2,  470.5, ...,  -75.7,  258.4,    0. ],
       [ 476.2,  760. ,  384.3, ...,  -56.9,  240.8,    0. ],
       [ 470.5,  384.3,  718. , ...,  -5